# ***음성도 결국 숫자일 뿐입니다***
- 먼저 카메라앱을 이용해 동영상을 만듭니다
- 만든 동영상으로부터 음성을 분리합니다
- 음성파일로부터 숫자를 추출하여 텍스트 파일로 만들어서 숫자가 어떻게 기록되어 있는지 확인합니다
- 숫자를 다시 음성파일로 만들고 재생해봅니다
  - 이 시점에서 음성 = 숫자 인정

In [ ]:
! pip install moviepy==1.0.3 numpy==2.1

## ***동영상으로부터 음성파일 분리하기***
- 아래 movie.mp4를 실제 동영상 파일의 위치로 바꿉니다
  - 카메라앱에서 만든 동영상 파일의 위치로 바꿉니다
- 동영상 파일로부터 audio.wav파일이 만들어집니다

In [ ]:
from moviepy import VideoFileClip, AudioFileClip, AudioArrayClip, vfx, concatenate_videoclips
import soundfile as sf
import numpy as np

clip = VideoFileClip("movie.mp4")
clip.audio.write_audiofile("audio.wav", fps=44100, nbytes=2)

## ***음성파일을 읽어서 숫자 확인하기***
- 음성파일을 읽었더니 숫자가 나옵니다
- 44100이 먼저 나옵니다
  - 1초에 44100개의 숫자 데이터가 있다는 뜻입니다
  - 이 숫자가 크면 좀더 음성 정보가 정밀해집니다
- 2는 스테레오 음성이라는 뜻입니다
  - 왼쪽, 오른쪽 

In [ ]:
data, sr = sf.read("audio.wav")

print(sr)
print(data.shape)
for d in data:
    print(d)

## ***음성 숫자를 텍스트 파일로 저장하기***
- 위에서 확인한 숫자가 텍스트 파일에 그대로 기록되었는지 확인해봅니다

In [ ]:
np.savetxt("audio.txt", data)

## ***텍스트 파일을 읽어서 숫자 확인하기***
- 텍스트 파일에 기록되었던 숫자가 다시 보이는지 확인해봅니다

In [ ]:
import numpy as np

data = np.loadtxt("audio.txt")

for d in data:
    print(d)

## ***읽은 숫자를 다시 음성파일로 만들기***
- 아래 44100은 원본 Sample Rate 입니다
- 1초게 44100개의 숫자 데이터가 있다는 뜻입니다
  - 원본 음성과 동일한 정보입니다
- restore.wav를 재생해봅니다
- ***이제 음성파일에 있던 정보는 그냥 숫자였다는 것을 알 수 있습니다***

In [ ]:
import soundfile as sf
import numpy as np

sf.write("restored.wav", data, 44100)

## ***모노 음성 만들기***
- 2개의 숫자 데이터 쌍을 1개로 줄입니다

In [ ]:
import soundfile as sf

data = np.loadtxt("audio.txt")
mono = data.mean(axis=1)
mono.shape

for d in mono:
    print(d)

- 모노 숫자를 모노 음성파이롤 만들고 이를 재생해봅니다

In [ ]:
sf.write(
    "mono.wav",
    mono,
    samplerate=44100
)


## ***샘플 줄이기***
- 원본 음성에는 1초에 44100개의 숫자쌍이 있었습니다
- 이것을 5분의 1로 줄여보겠습니다

In [ ]:
import soundfile as sf


#####################################
# 이 값을 바꿔보면서 반복 테스트 해봅니다 #
#####################################
downsample_ratio = 5

data = np.loadtxt("audio.txt")
downsampled = data[::downsample_ratio]
downsampled

- 원본과 비교하면 숫자의 규모가 5분의 1로 줄어든 것을 볼 수 있습니다
- 1초를 구성하는 숫자의 규모가 44100개에서 8820개로 줄어들게 됩니다

In [ ]:
data

In [ ]:
downsample_rate = 44100//downsample_ratio
downsample_rate

- 다운샘플링된 음성을 들어봅니다
- 위로 올라가서 downsample_ratio를 더 크게 바꾸고 다시 테스트해봅니다

In [ ]:
print("원본 길이:", len(data))
print("다운샘플 길이:", len(downsampled))

sf.write(
    "restore_downsample.wav",
    downsampled,
    samplerate=downsample_rate
)


## ***음성을 빠르게 재생하기***
- 숫자의 총량은 동일한데 1초를 구성하는 숫자의 규모를 늘리면 어떻게 될까요?
- 음성의 총 길이가 줄어듭니다 = 음성이 빠르게 재생됩니다

In [ ]:
import soundfile as sf


#####################################
# 이 값을 바꿔보면서 반복 테스트 해봅니다 #
#####################################
fast_ratio = 2

data = np.loadtxt("audio.txt")

fast_sr = round(44100 * fast_ratio)

sf.write(
    "restored_fast.wav",
    data,
    samplerate=fast_sr
)


## ***음성을 느리게 재생하기***
- 숫자의 총량은 동일한데 1초를 구성하는 숫자의 규모를 줄이면 어떻게 될까요?
- 음성의 총 길이가 늘어납니다 = 음성이 느리게 재생됩니다

In [ ]:
import soundfile as sf


#####################################
# 이 값을 바꿔보면서 반복 테스트 해봅니다 #
#####################################
slow_ratio = 1.5

data = np.loadtxt("audio.txt")

slow_sr = round(44100 / slow_ratio)

sf.write(
    "restored_slow.wav",
    data,
    samplerate=slow_sr
)


## ***음성 받아적기(Speech-To-Text)***
- OpenAI가 공개한 whisper라는 모델을 사용하면 비디오, 오디오 파일로부터 음성을 추출할 수 있습니다
- 이를 STT(Speath-To-Text) 라고 합니다
- 음성이 숫자였고 텍스트도 숫자이기 때문에 숫자를 숫자로 바꿔주는 AI인 것입니다

In [ ]:
! pip install openai-whisper

In [ ]:
import whisper

model = whisper.load_model("small")

def transcribe_audio(video_path):
    result = model.transcribe(video_path)

    timeline = []
    for seg in result["segments"]:
        timeline.append({
            "start": seg["start"],
            "end": seg["end"],
            "text": seg["text"]
        })

    return timeline


transcribe_audio('movie.mp4')